<a href="https://colab.research.google.com/github/monishramadoss/ofa_quant/blob/main/ofa_quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install ofa
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

fatal: destination path 'IMagenet' already exists and is not an empty directory.
test  train  val  wnids.txt  words.txt


In [10]:
import time
import skimage.io as nd
import numpy as np
import torch
import skimage.color


path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [nd.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), as_gray=False ) for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data += [nd.imread( path + 'val/images/{}'.format(img_name), as_gray=False)]
        
        # test_data.append(test_data_)
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    for i in range(len(test_data)):
        if test_data[i].ndim == 2:
            test_data[i] = skimage.color.gray2rgb(test_data[i])

    for i in range(len(train_data)):
        if train_data[i].ndim == 2:
            train_data[i] = skimage.color.gray2rgb(train_data[i])

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

def evaluate(model, data, target):
    model.eval()  # set model in eval mode
    total_time = 0
    num_correct = 0  # total 1000
    with torch.no_grad():
        for image, target in zip(data, target):
            # print(data[0].shape)
            start = time.time()
            result = model(image)
            total_time += time.time() - start
            
            prediction = idx2label[int(result[0].sort()[1][-1:])]
            if target == prediction:
                num_correct += 1
    
    inference_time = total_time / len(data)
    accuracy = num_correct / len(data)
    return inference_time, accuracy

train_data, train_labels, test_data, train_labels = get_data(get_id_dictionary())

starting loading data
finished loading data, in 74.54175019264221 seconds


In [11]:
print(train_data.shape, train_labels.shape)

(100000, 64, 64, 3) (10000, 200)


In [5]:
import copy
import torch
from ofa.model_zoo import ofa_net
from ofa.imagenet_classification.data_providers.imagenet import ImagenetDataProvider
# from ofa.imagenet_classification.run_manager import ImagenetRunConfig, RunManager

raw_resnet = ofa_net('ofa_resnet50', pretrained=True)
raw_resnet.set_max_net()
max_subnet = raw_resnet.get_active_subnet(preserve_weight=True)
raw_resnet.set_active_subnet(d=0, w=0, e=0)
min_subnet = raw_resnet.get_active_subnet(preserve_weight=True)


# ImagenetDataProvider.DEFAULT_PATH = path
# run_config = ImagenetRunConfig(test_batch_size=16, n_worker=20)
# run_config.data_provider.assign_active_img_size(64)


# run_manager = RunManager('./tmp/eval_subnet', min_subnet, run_config, init=False)
# run_manager.reset_running_statistics(net=min_subnet)

In [6]:
# loss, (top1, top5) = run_manager.validate(net=max_subnet, is_test=True)
# print('\nResults: loss=%.5f,\t top1=%.1f,\t top5=%.1f' % (loss, top1, top5))
# loss, (top1, top5) = run_manager.validate(net=min_subnet, is_test=True)
# print('\nResults: loss=%.5f,\t top1=%.1f,\t top5=%.1f' % (loss, top1, top5))

In [13]:
from ofa.imagenet_classification.networks import ResNets
from ofa.utils.layers import IdentityLayer, ResidualBlock

def val2list(val, repeat_time=1):
    if isinstance(val, list) or isinstance(val, np.ndarray):
        return val
    elif isinstance(val, tuple):
        return list(val)
    else:
        return [val for _ in range(repeat_time)]

def set_active_subnet(ofa, d=None, e=None, w=None, **kwargs):
    depth = val2list(d, len(ofa.BASE_DEPTH_LIST) + 1)
    expand_ratio = val2list(e, len(ofa.blocks))
    width_mult = val2list(w, len(ofa.BASE_DEPTH_LIST) + 2)
    for block, e in zip(ofa.blocks, expand_ratio):
        if e is not None:
            block.active_expand_ratio = e

    if width_mult[0] is not None:
        ofa.input_stem[1].conv.active_out_channel = ofa.input_stem[0].active_out_channel = \
            ofa.input_stem[0].out_channel_list[width_mult[0]]
    if width_mult[1] is not None:
        ofa.input_stem[2].active_out_channel = ofa.input_stem[2].out_channel_list[width_mult[1]]

    if depth[0] is not None:
        ofa.input_stem_skipping = (depth[0] != max(ofa.depth_list))
    for stage_id, (block_idx, d, w) in enumerate(zip(ofa.grouped_block_index, depth[1:], width_mult[2:])):
        if d is not None:
            ofa.runtime_depth[stage_id] = max(ofa.depth_list) - d
        if w is not None:
            for idx in block_idx:
                ofa.blocks[idx].active_out_channel = ofa.blocks[idx].out_channel_list[w]

def set_max_subnet(ofa):
    set_active_subnet(ofa, max(ofa.depth_list), max(ofa.expand_ratio_list), len(ofa.width_mult_list) - 1)

def get_active_subnet(ofa, preserve_weight=True):
    input_stem = [ofa.input_stem[0].get_active_subnet(3, preserve_weight)]
    active_out = ofa.input_stem[0].active_out_channel
    input_stem_blocks = [0]

    if ofa.input_stem_skipping <= 0:        
        input_stem.append(ResidualBlock(
            ofa.input_stem[1].conv.get_active_subnet(active_out, preserve_weight),
            IdentityLayer(active_out, active_out)
        ))
        input_stem_blocks += [1]
    input_stem_blocks += [2]
    input_stem.append(ofa.input_stem[2].get_active_subnet(active_out, preserve_weight))
    input_channel = ofa.input_stem[2].active_out_channel

    blocks = []
    block_groups = []
    for stage_id, block_idx in enumerate(ofa.grouped_block_index):
        depth_param = ofa.runtime_depth[stage_id]
        active_idx = block_idx[:len(block_idx) - depth_param]
        block_groups.append(active_idx)
        for idx in active_idx:
            blocks.append(ofa.blocks[idx].get_active_subnet(input_channel, preserve_weight))
            input_channel = ofa.blocks[idx].active_out_channel
    classifier = ofa.classifier.get_active_subnet(input_channel, preserve_weight)

    subnet = ResNets(input_stem, blocks, classifier)
    subnet.set_bn_param(**ofa.get_bn_param())
    subnet.input_stem_blocks = input_stem_blocks
    subnet.block_groups = block_groups
    return subnet


In [14]:
# run_manager_quant = RunManager('./tmp/eval', min_subnet, run_config=run_config, init=False, no_gpu=True)

In [15]:
input_fp32 = torch.randn(1, 3, 224, 224)

set_max_subnet(raw_resnet)
max_subnet = get_active_subnet(raw_resnet)

max_subnet.eval()
config = 'fbgemm'
max_subnet.qconfig = torch.quantization.get_default_qconfig(config)
max_subnet_1 = torch.quantization.prepare(max_subnet)
_ = max_subnet_1(input_fp32)
quat_max_model = torch.quantization.convert(max_subnet_1)




/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:123: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


In [16]:
temp = dict({('blocks.0.conv1.conv.scale', 0.3034),
 ('blocks.0.conv2.conv.scale', 0.3898),
 ('blocks.0.conv3.conv.scale', 0.3047),
 ('blocks.0.downsample.conv.scale', 0.4795),
 ('blocks.1.conv1.conv.scale', 0.2099),
 ('blocks.1.conv2.conv.scale', 0.2258),
 ('blocks.1.conv3.conv.scale', 0.3333),
 ('blocks.10.conv1.conv.scale', 0.2817),
 ('blocks.10.conv2.conv.scale', 0.3093),
 ('blocks.10.conv3.conv.scale', 0.2963),
 ('blocks.11.conv1.conv.scale', 0.6109),
 ('blocks.11.conv2.conv.scale', 0.5077),
 ('blocks.11.conv3.conv.scale', 0.3028),
 ('blocks.12.conv1.conv.scale', 0.2717),
 ('blocks.12.conv2.conv.scale', 0.2179),
 ('blocks.12.conv3.conv.scale', 0.1239),
 ('blocks.13.conv1.conv.scale', 0.2204),
 ('blocks.13.conv2.conv.scale', 0.4078),
 ('blocks.13.conv3.conv.scale', 0.3996),
 ('blocks.14.conv1.conv.scale', 0.2612),
 ('blocks.14.conv2.conv.scale', 0.2186),
 ('blocks.14.conv3.conv.scale', 0.3380),
 ('blocks.14.downsample.conv.scale', 0.5858),
 ('blocks.15.conv1.conv.scale', 0.7731),
 ('blocks.15.conv2.conv.scale', 0.4752),
 ('blocks.15.conv3.conv.scale', 0.5223),
 ('blocks.16.conv1.conv.scale', 0.3974),
 ('blocks.16.conv2.conv.scale', 0.2534),
 ('blocks.16.conv3.conv.scale', 0.3723),
 ('blocks.17.conv1.conv.scale', 0.2302),
 ('blocks.17.conv2.conv.scale', 0.2933),
 ('blocks.17.conv3.conv.scale', 0.3231),
 ('blocks.2.conv1.conv.scale', 0.1517),
 ('blocks.2.conv2.conv.scale', 0.1271),
 ('blocks.2.conv3.conv.scale', 0.1200),
 ('blocks.3.conv1.conv.scale', 0.1193),
 ('blocks.3.conv2.conv.scale', 0.1338),
 ('blocks.3.conv3.conv.scale', 0.0980),
 ('blocks.4.conv1.conv.scale', 0.2068),
 ('blocks.4.conv2.conv.scale', 0.3214),
 ('blocks.4.conv3.conv.scale', 0.4915),
 ('blocks.4.downsample.conv.scale', 0.2356),
 ('blocks.5.conv1.conv.scale', 0.1636),
 ('blocks.5.conv2.conv.scale', 0.3325),
 ('blocks.5.conv3.conv.scale', 0.3767),
 ('blocks.6.conv1.conv.scale', 0.2506),
 ('blocks.6.conv2.conv.scale', 0.2422),
 ('blocks.6.conv3.conv.scale', 0.1694),
 ('blocks.7.conv1.conv.scale', 0.2284),
 ('blocks.7.conv2.conv.scale', 0.3655),
 ('blocks.7.conv3.conv.scale', 0.3437),
 ('blocks.8.conv1.conv.scale', 0.3289),
 ('blocks.8.conv2.conv.scale', 0.3125),
 ('blocks.8.conv3.conv.scale', 0.3898),
 ('blocks.8.downsample.conv.scale', 0.5282),
 ('blocks.9.conv1.conv.scale', 0.3315),
 ('blocks.9.conv2.conv.scale', 0.3270),
 ('blocks.9.conv3.conv.scale', 0.2874),
 ('input_stem.0.conv.scale', 0.6389),
 ('input_stem.1.conv.conv.scale', 0.5264),
 ('input_stem.2.conv.scale', 0.5124)})

In [17]:
import torch.nn as nn
def nested_children(m: torch.nn.Module):
    children = dict(m.named_children())
    output = {}
    if children == {}:
        # if module has no children; m is last child! :O
        return m
    else:
        # look for children from children... to the last child!
        for name, child in children.items():
            try:
                output[name] = nested_children(child)
            except TypeError:
                output[name] = nested_children(child)
    return output

def squash_nested_dict(nested_dict, ret_lst={}, prefix='', mod=None):
    if nested_dict == {}:
        ret_lst[prefix[1:]] = mod
        return
    for k in nested_dict.keys():
        if isinstance(nested_dict[k], dict):
            squash_nested_dict(nested_dict[k], ret_lst, prefix+'.'+k)
        else:
            squash_nested_dict({}, ret_lst, prefix+'.'+k, nested_dict[k])



class Quant_Model(nn.Module):
    def __init__(self, quat_model, float_model):
        super(Quant_Model, self).__init__()
        self.input_zero_points = {}
        self.input_scales = {}
        self.float_model = float_model
        self.quat_model = quat_model
        self.quant_state_dict = quat_model.state_dict()

        layer_names = {}
        named_layers = nested_children(self.float_model)
        squash_nested_dict(named_layers, layer_names)
        
        for i, l in enumerate(list(layer_names.keys())):
            layer_names[l].register_forward_pre_hook(self.forward_pre_hook(l))

    def forward_pre_hook(self, layer_name):
        def pre_hook(module, x):
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Identity):
                with torch.no_grad():
                    zero_point = self.quant_state_dict[layer_name+'.zero_point'].float()
                    scale = self.quant_state_dict[layer_name+'.scale'].float()
                    quant_min = 0.0
                    quant_max = 1.0
                    tmp = torch.clamp(torch.round(torch.div(x[0], scale) + zero_point), quant_min, quant_max) - zero_point
                    x = tmp*scale
            return x
        return pre_hook

    def forward(self, x):
        return self.float_model(x)


set_active_subnet(raw_resnet, 0, 0, 0)
min_subnet = get_active_subnet(raw_resnet)
min_subnet.eval()
min_subnet.qconfig = torch.quantization.get_default_qconfig(config)

input_fp32 = torch.randn(1, 3, 224, 224, dtype=torch.float32)
min_subnet_1 = torch.quantization.prepare(min_subnet)

_ = min_subnet_1(input_fp32)
quat_min_model = torch.quantization.convert(min_subnet_1)

fake_quant_model = Quant_Model(quat_min_model, min_subnet)
fake_output = fake_quant_model(input_fp32)
print(fake_output)

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:123: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


tensor([[-294.3520,   -6.4059,  -54.2019,  -22.4575,  226.8641, -114.8154,
           48.1599,   56.9619,  178.0162,   37.6929,  -44.8707,  -82.7839,
           27.2446,   58.1014,  -16.6967,  -73.0909,  -38.6167,  -41.2054,
          -20.8798,  -46.9885, -116.7103,   21.6681, -210.4739,   35.4657,
           66.3895,  -70.4779,  -60.9834,  -41.7618,  -22.8354, -103.7682,
          194.7356,  -97.0490,   54.3800,  -55.0787,  -59.3974,   60.3609,
          131.6946,   79.7695, -164.7478,  114.2307,  -51.0119,   62.6533,
           56.7777,  141.8246,   72.9990,   -6.5813,  -47.0276,   84.5642,
           99.7476,  -23.6089,  114.4185,   18.6132, -103.0928,  -54.2114,
            1.4460, -149.8541, -155.3235,   81.7572,   33.7372,  -67.0638,
          -82.4987, -126.4726,  121.7566,   75.9322,  -99.2471,   99.3848,
         -124.1902,   -3.2048, -179.7852, -100.1188,  -33.5885,   -0.4123,
         -126.8417,  -46.3178,   -7.1046,  -75.6903,  -18.5515,   28.3253,
           -8.9110,  -30.

In [21]:
open("./tmp","w")
set_max_subnet(raw_resnet)

max_subnet = get_active_subnet(raw_resnet)

st_dict = torch.load('./weight_after_ps.pth')
print(st_dict.keys())
# max_subnet.load_state_dict(st_dict)
# max_subnet_1 = torch.quantization.prepare(max_subnet)

_ = max_subnet_1(input_fp32)
quat_max_model = torch.quantization.convert(max_subnet_1)

fake_quant_model = Quant_Model(quat_max_model, max_subnet)
fake_output = fake_quant_model(input_fp32)
print(fake_output)

odict_keys(['input_stem.0.conv.weight', 'input_stem.0.bn.weight', 'input_stem.0.bn.bias', 'input_stem.0.bn.running_mean', 'input_stem.0.bn.running_var', 'input_stem.0.bn.num_batches_tracked', 'input_stem.1.conv.conv.weight', 'input_stem.1.conv.bn.weight', 'input_stem.1.conv.bn.bias', 'input_stem.1.conv.bn.running_mean', 'input_stem.1.conv.bn.running_var', 'input_stem.1.conv.bn.num_batches_tracked', 'input_stem.2.conv.weight', 'input_stem.2.bn.weight', 'input_stem.2.bn.bias', 'input_stem.2.bn.running_mean', 'input_stem.2.bn.running_var', 'input_stem.2.bn.num_batches_tracked', 'blocks.0.conv1.conv.weight', 'blocks.0.conv1.bn.weight', 'blocks.0.conv1.bn.bias', 'blocks.0.conv1.bn.running_mean', 'blocks.0.conv1.bn.running_var', 'blocks.0.conv1.bn.num_batches_tracked', 'blocks.0.conv2.conv.weight', 'blocks.0.conv2.bn.weight', 'blocks.0.conv2.bn.bias', 'blocks.0.conv2.bn.running_mean', 'blocks.0.conv2.bn.running_var', 'blocks.0.conv2.bn.num_batches_tracked', 'blocks.0.conv3.conv.weight', 'blo

RuntimeError: ignored